In [ ]:
import logging
import boto3
from dateutil import parser as p
from dateutil.relativedelta import relativedelta as rd

In [ ]:
class accountCost:
    def __init__(self,session,mydate,mydays):
        
        try:
            # create client, fix region to us-east-1 as this is the only one that supports cost explorer
            client = session.client('ce', region_name='us-east-1')
            
        except Exception as e:
            logging.error(f'ERROR creating client for cost explorer: {e}')
        
        try:
            #convert input date from string
            end_date=p.parse(mydate)
            end_string= f'{end_date.year}-{end_date.month:02}-{end_date.day:02}'
        
            #calcualte yesterdays date
            start_date = (end_date + rd(days=mydays))
    
            #convert yesterdays date to string
            start_string= f'{start_date.year}-{start_date.month:02}-{start_date.day:02}'
        except Exception as e:
            logging.error(f'ERROR on reading date: {e}')
        
        try:
            #get spend from cost explorer api
            response = client.get_cost_and_usage(  
                TimePeriod={
                    'Start': start_string,
                    'End': end_string
                },
                Granularity="DAILY",
                Metrics=[
                    "UnblendedCost",
                ]
            )
        
            #return cost only from json api response, first value only
            logging.info(response['ResultsByTime'][0])
            
            total_cost=0.0
            
            for day in response['ResultsByTime']:
                total_cost=total_cost+float(day['Total']['UnblendedCost']['Amount'])
            
            self.cost=f'{total_cost:.2f}'
        
        except Exception as e:
            logging.error(f'ERROR on fetching data: {e}')
        
